<a href="https://colab.research.google.com/github/GreihMurray/NLP-2/blob/AG_Murray/irish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model
import joblib
import pickle

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
def read_file_to_sents():
    with open("train.tsv", encoding="utf-8") as file:
        f = csv.reader(file, delimiter="\t")

        cur_sent = []
        all_sents = []

        for line in tqdm(f, desc="Reading data..."):
            if line[0][0:2].strip() == 'N':
                line[0] = 'N'

            if line[0] == "<S>":
                if len(cur_sent) >= 1:
                    all_sents.append(cur_sent)
                cur_sent = []
                continue

            cur_sent.append((line[0], line[1]))

    return all_sents

In [4]:
def string_to_num(data):
    le = LabelEncoder()

    label = le.fit_transform(data)

    return label

In [5]:
def defeature(data):
    unfeatured = []

    for dict in data:
        unfeatured.append(dict['word'])

    return unfeatured

In [6]:
def seq_model(x, y):
    model = Sequential()
    model.add(Dense(256, input_shape=(1,), activation="sigmoid"))
    model.add(Dense(128, activation="sigmoid"))
    model.add(Dense(10, activation="softmax"))
    model.add(Dense(1, activation="softmax"))

  # From https://towardsdatascience.com/hyperparameter-tuning-with-kerastuner-and-tensorflow-c4a4d690b31a
    #stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

    x = defeature(x)

    x = string_to_num(x)
    y = string_to_num(y)

    x = np.asarray(x)
    y = np.asarray(y)

    print("[INFO] training network...")
    sgd = SGD(0.01)
    model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])
    H = model.fit(x[:10000], y[:10000], epochs=100, batch_size=100) #, callbacks=[stop_early])

    return model

In [7]:
def eval_model(model, x_test, y_test):
  x_test = defeature(x_test)
  x_test = string_to_num(x_test)
  y_test = string_to_num(y_test)
  return model.evaluate(x_test[:5000], y_test[:5000], return_dict=True)

Below work partially https://nlpforhackers.io/training-pos-tagger/amp/

In [8]:
def dec_tree(X, y):
    clf = Pipeline([
        ('vectorizer', DictVectorizer(sparse=False)),
        ('classifier', DecisionTreeClassifier(criterion='entropy')),
    ], verbose=1)

    print("Training Started")

    # Custom work below
    all_clfs = []

    for i in tqdm(range(0, 115), desc="Training"):
        cur_clf = clf
        cur_clf.fit(X[(i * 31649): ((i + 1) * 31649)], y[(i * 31649): ((i + 1) * 31649)])  # Use only the first 10K samples if you're running it multiple times. It takes a fair bit :)

        all_clfs.append(cur_clf)

        file_loc = '/content/gdrive/MyDrive/Colab_Notebooks/NLP/model' + str(i) + ".h5"
        file_loc_2 = '/gdrive/MyDrive/Colab_Notebooks/NLP/vector' + str(i) + ".pkl"

        # cur_clf.named_steps['classifier'].save(file_loc)

        with open(file_loc, 'wb') as pickle_file:
          pickle.dump(cur_clf, pickle_file)

        # cur_clf.named_steps['estimator'].model = None

        # joblib.dump(cur_clf, file_loc_2)

    print('Training completed')

    X_test, y_test = transform_to_dataset(test_sentences)

    return all_clfs

In [9]:
def acc_score(all_clfs, x_test, y_test):
    all_scores = []

    for clf in tqdm(all_clfs, desc="evaluating..."):
        all_scores.append(clf.score(x_test[:50000], y_test[:50000]))

    total_acc = 0

    for score in all_scores:
        total_acc += score

    print("Accuracy:", total_acc / len(all_scores))

In [10]:
# Below work comes from https://nlpforhackers.io/training-pos-tagger/amp/

In [11]:
def features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'is_capitalized': sentence[index][0].upper() == sentence[index][0],
        'is_all_caps': sentence[index].upper() == sentence[index],
        'is_all_lower': sentence[index].lower() == sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'has_hyphen': '-' in sentence[index],
        'is_numeric': sentence[index].isdigit(),
        'capitals_inside': sentence[index][1:].lower() != sentence[index][1:]
    }

In [12]:
# Below work comes from https://nlpforhackers.io/training-pos-tagger/amp/

In [13]:
def untag(tagged_sentence):
    return [w for w, t in tagged_sentence]

In [14]:
# Below work comes from https://nlpforhackers.io/training-pos-tagger/amp/

In [15]:
def transform_to_dataset(tagged_sentences):
    X, y = [], []

    for tagged in tagged_sentences:
        for index in range(len(tagged)):
            X.append(features(untag(tagged), index))
            y.append(tagged[index][1])

    return X, y

In [16]:
def load_models():
  all_clfs = []

  for i in range(1, 115):
    file_loc = '/content/gdrive/MyDrive/Colab_Notebooks/NLP/model' + str(i) + ".h5"

        # cur_clf.named_steps['classifier'].save(file_loc)

    with open(file_loc, 'rb') as pickle_file:
      cur_clf = pickle.load(pickle_file)

      all_clfs.append(cur_clf)

  return all_clfs



In [17]:
def anything():
    data = read_file_to_sents()

  # from https://nlpforhackers.io/training-pos-tagger/amp/
    cutoff = int(.75 * len(data))
    training_sentences = data[:cutoff]
    test_sentences = data[cutoff:]

    X, y = transform_to_dataset(training_sentences)

    del data, training_sentences

    x_test, y_test = transform_to_dataset(test_sentences)

  # Original
    #all_clfs = dec_tree(X, y)
    all_clfs = load_models()
    acc_score(all_clfs, x_test, y_test)

    #model = seq_model(X, y)
    #print(eval_model(model, x_test, y_test))

In [ ]:
anything()

Reading data...: 5057059it [00:06, 783986.25it/s]
evaluating...:  20%|██        | 23/114 [02:21<08:22,  5.52s/it]